## Scraping URLs

In [1]:
import urllib3
from urllib3 import Retry
from tqdm.notebook import tqdm
import time
from bs4 import BeautifulSoup

def scrape_html(df, path):
    html_content = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        if idx % 50 == 0 and idx!=0:
            time.sleep(10)
        url = row["web_url"]
        try:
            # html = urllib3.request("GET", url, retries=Retry(5))
            html = urllib3.request("GET", url, retries=False)
            if html.status == 200:
                soup = BeautifulSoup(markup = html.data, features='lxml')
                # kill all script and style elements
                for script in soup(["script", "style"]):
                    script.extract() # rip it out
                html_content.append(str(soup.body))
            else:
                html_content.append(None)
        except:
            html_content.append(None)
    df["html_content"] = html_content
    df.to_excel(path, index=False)

In [2]:
import pandas as pd

google = pd.read_excel("../data/Control_Google_results.xlsx")
scrape_html(df = google, path="../data/Control_Google_results.xlsx")

bing = pd.read_excel("../data/Control_Bing_results.xlsx")
scrape_html(df=bing, path="../data/Control_Bing_results.xlsx")

# google_reformed = pd.read_excel("../data/Google_QueryReformed_results.xlsx")
# scrape_html(df=google_reformed, path="../data/Google_QueryReformed_results.xlsx")

# bing_reformed = pd.read_excel("../data/Bing_QueryReformed_results.xlsx")
# scrape_html(df=bing_reformed, path="../data/Bing_QueryReformed_results.xlsx")

  0%|          | 0/2482 [00:00<?, ?it/s]

C:\Users\Hrishita Chakrabarti\AppData\Local\Temp\ipykernel_2816\1459555715.py:17: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(markup = html.data, features='lxml')


  0%|          | 0/2484 [00:00<?, ?it/s]

## Accessibility Markers

In [3]:
from bs4 import BeautifulSoup
import re

def get_text(content, url_flag):
    if url_flag == 1:
        
        soup = BeautifulSoup(content, features="lxml")
        # get text
        try:
            text = soup.body.get_text(separator='\n ', strip=True)
            num_list_items = len(soup.body.find_all("li"))
            num_headings = len(soup.body.find_all("h1")) + len(soup.body.find_all("h2")) + len(soup.body.find_all("h3")) + len(soup.body.find_all("h4")) + len(soup.body.find_all("h5")) + len(soup.body.find_all("h6"))
            num_paras = 0
            para_len = 0
            num_inpara_headings = 0
            for para in soup.body.find_all("p"):
                para_len += len(para.text.split())
                num_paras += 1
            avg_para_len = para_len/num_paras
                    
        except:
            return None
        return text, num_list_items, num_headings, num_inpara_headings, num_paras, avg_para_len
    if url_flag == 0:
        text = content
        text = text.strip()
        text = re.sub("\n{2,}", "\n", text)
        # print(text)
        lines = text.split("\n")
        # print(lines)
        num_list_items = 0
        num_headings = 0

        for line in lines:
            enum_match = re.search("\d+\.\s", line)
            list_match = re.search("^\*\s", line)
            if enum_match != None or list_match != None:
                num_list_items += 1

        for line in lines:
            heading_match = re.search("^\*{2}(?:[^\*]*)\*{2}", line) # **heading**
            if heading_match != None:
                # print("heading match ", line)
                num_headings += 1

        num_paras = 0
        para_len = 0
        num_inpara_headings = 0
        for line in lines:
            enum_match = re.search("\d+\.\s", line)
            list_match = re.search("^\*\s", line)
            if enum_match == None and list_match == None:
                heading_match = re.search("^\*{2}(?:[^\*]*)\*{2}", line)
                if heading_match == None:
                    para_len += len(line.split())
                    num_paras += 1
                else:
                    line = re.sub("^\*{2}(?:[^\*]*)\*{2}", "", line)
                    if line != "":
                        para_len += len(line.split())
                        num_inpara_headings += 1
                        num_paras += 1
        if num_paras != 0:
            avg_para_len = para_len/num_paras
        else:
            avg_para_len = 0

        return text, num_list_items, num_headings, num_inpara_headings, num_paras, avg_para_len

In [4]:
import syllables
import contractions
import nltk
import spacy
import re
import pandas as pd
import textstat

concrete_df = pd.read_excel("../Data/concreteness_data.xlsx", index_col='Word') # 1 - maximally abstract, 5 - maximally concrete
concrete_dict = concrete_df.to_dict('index')
bigram_words = [word for word in concrete_dict.keys() if len(str(word).split())>1]
# print(bigram_words)

nlp = spacy.load("en_core_web_sm")

def calc_concreteness(text):
    sum_concrete = 0
    concrete_count = 0
    abstract_count = 0

    for bigram in bigram_words:
        if bigram in text:
            num_occur = len([m.start() for m in re.finditer(bigram, text)])
            text = re.sub(bigram, "", text)
            concreteness = concrete_dict[bigram]["Conc.M"]
            if round(concreteness) < 4:
                abstract_count += num_occur
            else:
                concrete_count += num_occur
            sum_concrete += (concreteness * num_occur)

    for word in nlp(text):
        if concrete_dict.get(word.text, None) != None:
            concreteness = concrete_dict[word.text]['Conc.M']
            if concreteness < 4:
                abstract_count += 1
            else:
                concrete_count += 1
            sum_concrete += concreteness
    if concrete_count + abstract_count == 0:
        avg_concreteness = 0
    else:
        avg_concreteness = round(sum_concrete/(abstract_count + concrete_count), 2)
    concrete_ratio = round(concrete_count/len(nlp(text)), 2)
    abstract_ratio = round(abstract_count/len(nlp(text)), 2)
    return avg_concreteness, concrete_ratio, abstract_ratio  

def flesch_reading_ease(text):
    return round(textstat.flesch_reading_ease(text),2)
    # num_words = 0
    # num_sentences = 0
    # num_syllables = 0

    # text = text.lower()
    # sentences = nltk.sent_tokenize(text)
    # num_sentences = len(sentences)
    # for sentence in sentences:
    #     words = sentence.split(" ")
    #     num_words += len(words)
    #     for word in words:
    #         num_syllables += syllables.estimate(word)

    # score = 206.8835 - (1.015*(num_words/num_sentences)) - (84.6 * (num_syllables/num_words))

    # return round(score, 2)

def coleman_liau(text):

    return round(textstat.coleman_liau_index(text),2)

    # num_letters = 0
    # num_words = 0
    # num_sentences = 0

    # sentences = nltk.sent_tokenize(text)
    # num_sentences = len(sentences)
    # for sentence in sentences:
    #     words = sentence.split(" ")
    #     num_words += len(words)
    #     for word in words:
    #         num_letters += len(word)

    # L = (num_letters/num_words)*100
    # S = (num_sentences/num_words)*100

    # score = (0.0588 * L) - (0.296 * S) - 15.8

    # return round(score, 2)

def text_cleaning(text):
    text = re.sub('\\n{2,}', '\\n', text)
    text = re.sub(':', "\n", text)
    text = re.sub('\\t', ' ', text)
    # text = re.sub('\*+(?:\S+)\*+', '', text)
    # text = re.sub('\*', '', text)
    text = text.strip()
    text = re.sub('[\?\!\n]+', '. ', text)
    text = re.sub('[^A-Za-z0-9\s\-\_\.]', '', text)
    text = re.sub('\.{2,}', '. ', text) # ... -> .
    text = re.sub('\s{2,}', ' ', text) # multiple spaces -> singular space
    return text

def text_analysis(text):
    text = text_cleaning(text)
    if text != "":
        sentences = nltk.sent_tokenize(text)
        num_words = [len(sentence.split()) for sentence in sentences]
        avg_len = round(sum(num_words)/len(sentences),2)
        text = contractions.fix(text)
        avg_concreteness, concrete_ratio, abstract_ratio = calc_concreteness(text)
        flesch = flesch_reading_ease(text)
        cli = coleman_liau(text)

        return [
            text,
            len(sentences),
            avg_len,
            flesch,
            cli,
            avg_concreteness,
            concrete_ratio,
            abstract_ratio,
        ]
    else:
        return None

c:\Users\Hrishita Chakrabarti\anaconda3\envs\WebAccessibility\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Hrishita Chakrabarti\anaconda3\envs\WebAccessibility\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [8]:
# get_text(data["web_url"].iloc[0], url_flag=1)

In [5]:
from tqdm.notebook import tqdm

def compute_markers(data, file_name, data_type):
    default = [None, None, None, None, None, None, None]
    ratio_list_items = None
    ratio_headings = None
    ratio_paras = None
    avg_para_len = None

    path = "../results/" + file_name + ".xlsx"

    if data_type == "SE_RR": # SE_RR
        column_names = ["query", "query_category", "web_title", "web_url", "cleaned_text", "ratio_paras", "ratio_list_items", "ratio_headings", "avg_para_len", "num_sentences", "avg_len", "flesch", "cli", "avg_concrete", "concrete_ratio", "abstract_ratio"]
    elif data_type == "SE_SERP": # SE_SERP
        column_names = ["query", "query_category", "web_title", "web_snippet",  "cleaned_text", "ratio_paras", "ratio_list_items", "ratio_headings", "avg_para_len", "num_sentences", "avg_len", "flesch", "cli", "avg_concrete", "concrete_ratio", "abstract_ratio"]
    elif data_type == "GenAI": # GenAI
        column_names = ["query", "query_category", "response_text", "cleaned_text", "ratio_paras", "ratio_list_items", "ratio_headings", "avg_para_len", "num_sentences", "avg_len", "flesch", "cli", "avg_concrete", "concrete_ratio", "abstract_ratio"]
    results = []

    for idx, row in tqdm(data.iterrows(), total=len(data)):
        if data_type == "SE_RR":
            text_results = get_text(row["html_content"], url_flag = 1)
        elif data_type == "SE_SERP":
            web_title = row["web_title"]
            web_snippet = row["web_snippet"]
            text_results = get_text("\n".join([web_title, web_snippet]), url_flag = 0)
        elif data_type == "GenAI":
            text_results = get_text(row["response_text"], url_flag = 0)
        
        if text_results != None:
            text, num_list_items, num_headings, num_inpara_headings, num_paras, avg_para_len = text_results
            if data_type == "SE_SERP":
                num_headings += 1 # web title is counted as a heading for web snippets

            scores = text_analysis(text)

            if scores == None:
                scores = default
            else:
                ratio_list_items = num_list_items/scores[1]
                ratio_headings = num_headings/scores[1]
                ratio_paras = 1 - (ratio_list_items + ratio_headings) + num_inpara_headings/scores[1]

        else:
            scores = default

        if data_type == "SE_RR":
            result = [row["query"], row["query_category"], row["web_title"], row["web_url"]] + [scores[0]] + [ratio_paras, ratio_list_items, ratio_headings, avg_para_len] + scores[1:]
        elif data_type == "SE_SERP":
            result = [row["query"], row["query_category"], row["web_title"], row["web_snippet"]] + [scores[0]] + [ratio_paras, ratio_list_items, ratio_headings, avg_para_len] + scores[1:]
        elif data_type == "GenAI":
            result = [row["query"], row["query_category"], row["response_text"]] + [scores[0]] + [ratio_paras, ratio_list_items, ratio_headings, avg_para_len] + scores[1:]

        results.append(result)
        
        if idx % 500 == 0 and idx!=0:
            results_df = pd.DataFrame(results, columns = column_names)
            results_df.to_excel(path, index=False)

    results_df = pd.DataFrame(results, columns = column_names)
    results_df.to_excel(path, index=False)

In [6]:
import pandas as pd

google = pd.read_excel("../Data/Control_Google_results.xlsx").dropna()
bing = pd.read_excel("../Data/Control_Bing_results.xlsx").dropna()
gemini = pd.read_excel("../Data/Control_Gemini_results.xlsx").dropna()
gpt = pd.read_excel("../Data/Control_ChatGPT_results.xlsx").dropna()

# google_reformed = pd.read_excel("../Data/Google_QueryReformed_results.xlsx").dropna()
# bing_reformed = pd.read_excel("../Data/Bing_QueryReformed_results.xlsx").dropna()
# gemini_reformed = pd.read_excel("../Data/Gemini_QueryReformed_results.xlsx").dropna()
# gpt_reformed = pd.read_excel("../Data/ChatGPT_QueryReformed_results.xlsx").dropna()

In [7]:
compute_markers(data = google, file_name = "Control_Google_RR_results", data_type="SE_RR")
compute_markers(data = google, file_name = "Control_Google_SERP_results", data_type="SE_SERP")
compute_markers(data = bing, file_name = "Control_Bing_RR_results", data_type="SE_RR")
compute_markers(data = bing, file_name = "Control_Bing_SERP_results", data_type="SE_SERP")
compute_markers(data = gemini, file_name = "Control_Gemini_results", data_type="GenAI")
compute_markers(data = gpt, file_name = "Control_ChatGPT_results", data_type="GenAI")

# compute_markers(data = google_reformed, file_name = "Google_QueryReformed_RR_results", data_type="SE_RR")
# compute_markers(data = google_reformed, file_name = "Google_QueryReformed_SERP_results", data_type="SE_SERP")
# compute_markers(data = bing_reformed, file_name = "Bing_QueryReformed_RR_results", data_type="SE_RR")
# compute_markers(data = bing_reformed, file_name = "Bing_QueryReformed_SERP_results", data_type="SE_SERP")
# compute_markers(data = gemini_reformed, file_name = "Gemini_QueryReformed_results", data_type="GenAI")
# compute_markers(data = gpt_reformed, file_name = "ChatGPT_QueryReformed_results", data_type="GenAI")

  0%|          | 0/1674 [00:00<?, ?it/s]

  0%|          | 0/1674 [00:00<?, ?it/s]

  0%|          | 0/1708 [00:00<?, ?it/s]

  0%|          | 0/1708 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

## Merging Results

In [1]:
import pandas as pd

google_RR = pd.read_excel("../results/Google_RR_results.xlsx")
google_snip = pd.read_excel("../results/Google_SERP_results.xlsx")
bing_RR = pd.read_excel("../results/Bing_RR_results.xlsx")
bing_snip = pd.read_excel("../results/Bing_SERP_results.xlsx")
gemini = pd.read_excel("../results/Gemini_results.xlsx")
gpt = pd.read_excel("../results/ChatGPT_results.xlsx")

google_RR_control = pd.read_excel("../results/Control_Google_RR_results.xlsx")
google_snip_control = pd.read_excel("../results/Control_Google_SERP_results.xlsx")
bing_RR_control = pd.read_excel("../results/Control_Bing_RR_results.xlsx")
bing_snip_control = pd.read_excel("../results/Control_Bing_SERP_results.xlsx")
gemini_control = pd.read_excel("../results/Control_Gemini_results.xlsx")
gpt_control = pd.read_excel("../results/Control_ChatGPT_results.xlsx")

google_reformed_RR = pd.read_excel("../results/Google_QueryReformed_RR_results.xlsx")
google_reformed_snip = pd.read_excel("../results/Google_QueryReformed_SERP_results.xlsx")
bing_reformed_RR = pd.read_excel("../results/Bing_QueryReformed_RR_results.xlsx")
bing_reformed_snip = pd.read_excel("../results/Bing_QueryReformed_SERP_results.xlsx")
gemini_reformed = pd.read_excel("../results/Gemini_QueryReformed_results.xlsx")
gpt_reformed = pd.read_excel("../results/ChatGPT_QueryReformed_results.xlsx")

# queries = pd.read_excel("../data/SelectedQueries_2.xlsx")

In [2]:
import re

def clean_df(df):
    df = df.dropna(how='any', inplace=False)
    df = df.drop(df[df.num_sentences <=0].index, inplace=False)
    df = df.drop(df[df.avg_len <=0].index, inplace=False)
    # df = df.drop(df[df.flesch <=0].index, inplace=False)
    # df = df.drop(df[df.cli <=0].index, inplace=False)
    # df = df.drop(df[df.avg_concrete <=0].index, inplace=False)
    # df = df.drop(df[df.concrete_ratio <=0].index, inplace=False)
    # df = df.drop(df[df.abstract_ratio <=0].index, inplace=False)

    return df

def merge_results(df, file_name, response_group_name, response_group_type, control=0):
    if control == 0:
        queries = pd.read_excel("../data/SelectedQueries_2.xlsx")
    else:
        queries = pd.read_excel("../data/ControlQueries.xlsx")
    df = clean_df(df)
    marker_columns = ["ratio_paras", "ratio_list_items", "ratio_headings", "avg_para_len", "num_sentences", "avg_len", "flesch", "cli", "avg_concrete", "concrete_ratio", "abstract_ratio"]
    
    df_merged = pd.DataFrame(
        df.groupby(["query"]).mean(numeric_only=True).values.tolist(),
        columns = marker_columns
    )

    df_merged = df_merged.round(2)
    df_merged.insert(0, "query", df.groupby(["query"]).mean(numeric_only=True).index.tolist())
    ngrams = []
    domain_relevant = []
    for query in df_merged["query"]:
        if len(query.split())>4:
            ngrams.append(">4")
        else:
            ngrams.append(str(len(query.split())))
        if control == 0:
            domain_relevant.append(queries.loc[queries['query']==query]['domain_relevant'].tolist()[0])
    df_merged.insert(1, "ngram", ngrams, True)
    if control == 0:
        df_merged.insert(2, "domain_specific", domain_relevant, True)
    df_merged.insert(len(df_merged.columns), "undefined_ratio", 1-(df_merged["concrete_ratio"] + df_merged["abstract_ratio"]), True)

    present_queries = df_merged["query"].tolist()
    all_queries = queries["query"].tolist()

    for query in all_queries:
        if query not in present_queries:
            marker_values = [0] * len(marker_columns)
            ngram = len(query.split())
            if ngram > 4:
                ngram = ">4"
            else:
                ngram = str(ngram)
            if control == 0:
                domain_relevant = queries.loc[queries['query']==query]['domain_relevant'].tolist()[0]   
            undefined_ratio = 0
            if control == 0:
                row = [query, ngram, domain_relevant] + marker_values + [undefined_ratio]
            else:
                row = [query, ngram] + marker_values + [undefined_ratio]
            df_merged.loc[len(df_merged), df_merged.columns] = row     

    df_merged['response_group'] = [response_group_name] * len(df_merged)
    df_merged['response_group_type'] = [response_group_type] * len(df_merged)

    path = '../results/'+file_name
    df_merged.to_excel(path, index=False)

merge_results(google_RR, "Google_RR_merged.xlsx", response_group_name='Google RR', response_group_type='RR')
merge_results(google_snip, "Google_SERP_merged.xlsx", response_group_name='Google SERP', response_group_type='SERP')
merge_results(bing_RR, "Bing_RR_merged.xlsx", response_group_name='Bing RR', response_group_type='RR')
merge_results(bing_snip, "Bing_SERP_merged.xlsx", response_group_name='Bing SERP', response_group_type='SERP')
merge_results(gemini, "Gemini_merged.xlsx", response_group_name='Gemini', response_group_type='Chatbot')
merge_results(gpt, "ChatGPT_merged.xlsx", response_group_name='GPT 3.5', response_group_type='Chatbot')

merge_results(google_RR_control, "Control_Google_RR_merged.xlsx", response_group_name='Google RR', response_group_type='RR', control=1)
merge_results(google_snip_control, "Control_Google_SERP_merged.xlsx", response_group_name='Google SERP', response_group_type='SERP', control=1)
merge_results(bing_RR_control, "Control_Bing_RR_merged.xlsx", response_group_name='Bing RR', response_group_type='RR', control=1)
merge_results(bing_snip_control, "Control_Bing_SERP_merged.xlsx", response_group_name='Bing SERP', response_group_type='SERP', control=1)
merge_results(gemini_control, "Control_Gemini_merged.xlsx", response_group_name='Gemini', response_group_type='Chatbot', control=1)
merge_results(gpt_control, "Control_ChatGPT_merged.xlsx", response_group_name='GPT 3.5', response_group_type='Chatbot', control=1)

merge_results(google_reformed_RR, "Google_QueryReformed_RR_merged.xlsx", response_group_name='Google RR Reformed', response_group_type='RR Reformed')
merge_results(google_reformed_snip, "Google_QueryReformed_SERP_merged.xlsx", response_group_name='Google SERP Reformed', response_group_type='SERP Reformed')
merge_results(bing_reformed_RR, "Bing_QueryReformed_RR_merged.xlsx", response_group_name='Bing RR Reformed', response_group_type='RR Reformed')
merge_results(bing_reformed_snip, "Bing_QueryReformed_SERP_merged.xlsx", response_group_name='Bing SERP Reformed', response_group_type='SERP Reformed')
merge_results(gemini_reformed, "Gemini_QueryReformed_merged.xlsx", response_group_name='Gemini Reformed', response_group_type='Chatbot Reformed')
merge_results(gpt_reformed, "ChatGPT_QueryReformed_merged.xlsx", response_group_name='GPT 3.5 Reformed', response_group_type='Chatbot Reformed')

## Descriptive Stats

### General comparison

In [3]:
import numpy as np

def gen_comparison_desc_stats(merged_df, file_name):
    markers = [
            "ratio_paras",
            "ratio_list_items",
            "ratio_headings",
            "avg_para_len",
            "num_sentences",
            "avg_len",
            "flesch",
            "cli",
            "avg_concrete",
            "concrete_ratio",
            "abstract_ratio",
            "undefined_ratio"
        ]

    response_group_types = [
        "SERP",
        "RR",
        "Chatbot"
    ]

    response_groups = [
        "Google SERP",
        "Google RR",
        "Bing SERP",
        "Bing RR",
        "Gemini",
        "GPT 3.5"
    ]

    results = []

    for response_group_type in response_group_types:
        df = merged_df.loc[merged_df["response_group_type"]==response_group_type]
        for marker in markers:
            mean = round(np.mean(df[marker].to_numpy()), 2)
            std = round(np.std(df[marker].to_numpy()), 2)
            median = round(np.median(df[marker].to_numpy()), 2)
            results.append([response_group_type, marker, mean, std, median])

    for response_group in response_groups:
        df = merged_df.loc[merged_df["response_group"]==response_group]
        for marker in markers:
            mean = round(np.mean(df[marker].to_numpy()), 2)
            std = round(np.std(df[marker].to_numpy()), 2)
            median = round(np.median(df[marker].to_numpy()), 2)
            results.append([response_group, marker, mean, std, median])


    desc_stats_gen_comparison = pd.DataFrame(results, columns=["group_name", "marker", "mean", "std", "median"])
    desc_stats_gen_comparison.to_excel("../results/" + file_name, index=False)

In [4]:
import pandas as pd

bing_RR = pd.read_excel("../results/Bing_RR_merged.xlsx")
bing_snip = pd.read_excel("../results/Bing_SERP_merged.xlsx")
google_RR = pd.read_excel("../results/Google_RR_merged.xlsx")
google_snip = pd.read_excel("../results/Google_SERP_merged.xlsx")
gemini = pd.read_excel("../results/gemini_merged.xlsx")
gpt = pd.read_excel("../results/ChatGPT_merged.xlsx")

target_merged_df = pd.concat([google_snip, google_RR, bing_snip, bing_RR, gemini, gpt], ignore_index=True)


bing_RR_control = pd.read_excel("../results/Control_Bing_RR_merged.xlsx")
bing_snip_control = pd.read_excel("../results/Control_Bing_SERP_merged.xlsx")
google_RR_control = pd.read_excel("../results/Control_Google_RR_merged.xlsx")
google_snip_control = pd.read_excel("../results/Control_Google_SERP_merged.xlsx")
gemini_control = pd.read_excel("../results/Control_gemini_merged.xlsx")
gpt_control = pd.read_excel("../results/Control_ChatGPT_merged.xlsx")
control_merged_df = pd.concat([google_snip_control, google_RR_control, bing_snip_control, bing_RR_control, gemini_control, gpt_control], ignore_index=True)

In [5]:
gen_comparison_desc_stats(target_merged_df, "DescriptiveStatsGenComparison.xlsx")
gen_comparison_desc_stats(control_merged_df, "Control_DescriptiveStatsGenComparison.xlsx")

### Query Category

In [6]:
merged_df = target_merged_df

In [7]:
import numpy as np

markers = [
        "ratio_paras",
        "ratio_list_items",
        "ratio_headings",
        "avg_para_len",
        "num_sentences",
        "avg_len",
        "flesch",
        "cli",
        "avg_concrete",
        "concrete_ratio",
        "abstract_ratio",
        "undefined_ratio"
    ]

response_groups = [
    "Google SERP",
    "Google RR",
    "Bing SERP",
    "Bing RR",
    "Gemini",
    "GPT 3.5"
]

results = []

for response_group in response_groups:
    for ngram in ["1", "2", "3", "4", ">4"]:
        df = merged_df.loc[(merged_df["response_group"]==response_group) & (merged_df['ngram']==ngram)]
        if df.empty:
            print(ngram, response_group)
        for marker in markers:
            mean = round(np.mean(df[marker].to_numpy()), 2)
            std = round(np.std(df[marker].to_numpy()), 2)
            median = round(np.median(df[marker].to_numpy()), 2)
            results.append([response_group, str(ngram) + "-gram", marker, mean, std, median])
    
    for domain in [True, False]:
        df = merged_df.loc[(merged_df["response_group"]==response_group) & (merged_df['domain_specific']==domain)]
        if df.empty:
            print(domain, response_group)
        for marker in markers:
            mean = round(np.mean(df[marker].to_numpy()), 2)
            std = round(np.std(df[marker].to_numpy()), 2)
            median = round(np.median(df[marker].to_numpy()), 2)
            if domain == True:
                category = "domain specific"
            else:
                category = "general"
            results.append([response_group, category, marker, mean, std, median])


desc_stats_query_category = pd.DataFrame(results, columns=["group_name", "query_category", "marker", "mean", "std", "median"])

In [8]:
desc_stats_query_category.to_excel("../results/DescriptiveStatsQueryCategory.xlsx", index=False)

### Query Reformulation

In [1]:
import pandas as pd

bing_RR = pd.read_excel("../results/Bing_RR_merged.xlsx")
bing_snip = pd.read_excel("../results/Bing_SERP_merged.xlsx")
google_RR = pd.read_excel("../results/Google_RR_merged.xlsx")
google_snip = pd.read_excel("../results/Google_SERP_merged.xlsx")
gemini = pd.read_excel("../results/gemini_merged.xlsx")
gpt = pd.read_excel("../results/ChatGPT_merged.xlsx")
google_reformed_RR = pd.read_excel("../results/Google_QueryReformed_RR_merged.xlsx")
google_reformed_snip = pd.read_excel("../results/Google_QueryReformed_SERP_merged.xlsx")
bing_reformed_RR = pd.read_excel("../results/Bing_QueryReformed_RR_merged.xlsx")
bing_reformed_snip = pd.read_excel("../results/Bing_QueryReformed_SERP_merged.xlsx")
gemini_reformed = pd.read_excel("../results/Gemini_QueryReformed_merged.xlsx")
gpt_reformed = pd.read_excel("../results/ChatGPT_QueryReformed_merged.xlsx")

merged_df = pd.concat([google_snip, google_RR, bing_snip, bing_RR, gemini, gpt, google_reformed_snip, google_reformed_RR, bing_reformed_snip, bing_reformed_RR, gemini_reformed, gpt_reformed], ignore_index=True)

In [2]:
import numpy as np

markers = [
        "ratio_paras",
        "ratio_list_items",
        "ratio_headings",
        "avg_para_len",
        "num_sentences",
        "avg_len",
        "flesch",
        "cli",
        "avg_concrete",
        "concrete_ratio",
        "abstract_ratio",
        "undefined_ratio"
    ]

response_group_types = [
    ["SERP", "SERP Reformed"],
    ["RR", "RR Reformed"],
    ["Chatbot", "Chatbot Reformed"],
]

response_groups = [
    ["Google SERP", "Google SERP Reformed"],
    ["Google RR", "Google RR Reformed"],
    ["Bing SERP", "Bing SERP Reformed"],
    ["Bing RR", "Bing RR Reformed"],
    ["Gemini", "Gemini Reformed"],
    ["GPT 3.5", "GPT 3.5 Reformed"],
]

In [3]:
results = []

for response_group_type in response_group_types:
    original = response_group_type[0]
    reformed = response_group_type[1]
    df_original = merged_df.loc[merged_df["response_group_type"] == original]
    df_reformed = merged_df.loc[merged_df["response_group_type"] == reformed]
    for marker in markers:
        mean_diff = round(np.mean(df_reformed[marker].to_numpy()) - np.mean(df_original[marker].to_numpy()), 2)
        std_diff = round(np.std(df_reformed[marker].to_numpy()) - np.std(df_original[marker].to_numpy()), 2)
        median_diff = round(np.median(df_reformed[marker].to_numpy()) - np.median(df_original[marker].to_numpy()), 2)
        results.append([response_group_type[0], marker, mean_diff, std_diff, median_diff])

for response_group in response_groups:
    original = response_group[0]
    reformed = response_group[1]
    df_original = merged_df.loc[merged_df["response_group"] == original]
    df_reformed = merged_df.loc[merged_df["response_group"] == reformed]
    for marker in markers:
        mean_original = np.mean(df_original[marker].to_numpy())
        mean_reformed = np.mean(df_reformed[marker].to_numpy()) 
        std_original = np.std(df_original[marker].to_numpy())
        std_reformed = np.std(df_reformed[marker].to_numpy())
        median_original = np.median(df_original[marker].to_numpy())
        median_reformed = np.median(df_reformed[marker].to_numpy())
        results.append([response_group[0], marker, mean_original, std_original, median_original, mean_reformed, std_reformed, median_reformed])


desc_stats_query_reformulation = pd.DataFrame(results, columns=["group_name", "marker", "mean_original", "std_original", "median_original", "mean_reformed", "std_reformed", "median_reformed"])
desc_stats_query_reformulation.to_excel("../results/DescriptiveStatsQueryReformulation.xlsx", index=False)

In [8]:
results = []

for response_group_type in response_group_types:
    original = response_group_type[0]
    reformed = response_group_type[1]
    df_original = merged_df.loc[merged_df["response_group_type"] == original]
    df_reformed = merged_df.loc[merged_df["response_group_type"] == reformed]
    for marker in markers:
        mean_diff = round(np.mean(df_reformed[marker].to_numpy()) - np.mean(df_original[marker].to_numpy()), 2)
        std_diff = round(np.std(df_reformed[marker].to_numpy()) - np.std(df_original[marker].to_numpy()), 2)
        median_diff = round(np.median(df_reformed[marker].to_numpy()) - np.median(df_original[marker].to_numpy()), 2)
        results.append([response_group_type[0], marker, mean_diff, std_diff, median_diff])

for response_group in response_groups:
    original = response_group[0]
    reformed = response_group[1]
    df_original = merged_df.loc[merged_df["response_group"] == original]
    df_reformed = merged_df.loc[merged_df["response_group"] == reformed]
    for marker in markers:
        mean_diff = round(np.mean(df_reformed[marker].to_numpy()) - np.mean(df_original[marker].to_numpy()), 2)
        std_diff = round(np.std(df_reformed[marker].to_numpy()) - np.std(df_original[marker].to_numpy()), 2)
        median_diff = round(np.median(df_reformed[marker].to_numpy()) - np.median(df_original[marker].to_numpy()), 2)
        results.append([response_group[0], marker, mean_diff, std_diff, median_diff])


desc_stats_query_reformulation = pd.DataFrame(results, columns=["group_name", "marker", "mean_diff", "std_diff", "median_diff"])
desc_stats_query_reformulation.to_excel("../results/DescriptiveStatsQueryReformulation_Difference.xlsx", index=False)